Data was published at https://github.com/duyvuleo/VNTC

# Introduction

In this tutorial, we will implement some algorithms to apply in text summarization problem.

## What is Text Summarization?

Text summarization is the problem of creating a short, accurate, and fluent summary of a longer text document.

Automatic text summarization methods are greatly needed to address the ever-growing amount of text data available online to both better help discover relevant information and to consume relevant information faster.

## What will we do in this tutorial?

In this tutorial, we will solve Text Summarization for Vietnamese newspapers, using some algorithms belows:
1. Extractive Text Summarization
    - Doc2Vec
    - Text Rank
2. Abstractive Text Summarization
    - Google textsum


We just implement "**Single document summarization**" problem in this tutorial, another problem called "**Multi-document summarization**" will be dicussed in another time.

# Extractive Text Summarization

## Doc2Vec

example: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb

### Basic idea
The idea of using Doc2Vec algorithm for text summarization problem is described as follows:
1. In all documents, we will extract sentences separately.
2. Each sentence will be represented by a vector, via doc2vec model
3. Use KMean algorithm to find out most featured sentences.

In [1]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from pyvi import ViTokenizer, ViPosTagger
from tqdm import tqdm
import numpy as np
import gensim
import numpy as np

In [3]:
import os 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
dir_path = os.path.join(dir_path, 'Data')

sentences = []

In [4]:
import pickle

def get_data(folder):
    sentences = []
    for path in os.listdir(folder):
        file_path = os.path.join(folder, path)
        with open(file_path, 'r', encoding="utf-16") as f:

            lines = f.readlines()

            for line in lines:
                sens = line.split('.')
                for sen in sens:
                    if len(sen) > 10:
                        sen = gensim.utils.simple_preprocess(sen)
                        sen = ' '.join(sen)
                        sen = ViTokenizer.tokenize(sen)
                        sentences.append(sen)

    return sentences

You can use multiprocessing here, but we will not use it for easy in understanding code.

In [5]:
# from multiprocessing import Pool
# sentences = []
train_paths = [os.path.join(dir_path, 'VNTC-master/Data/10Topics/Ver1.1/Train_Full'), 
               os.path.join(dir_path, 'VNTC-master/Data/10Topics/Ver1.1/Test_Full'),
               os.path.join(dir_path, 'VNTC-master/Data/27Topics/Ver1.1/new train'),
               os.path.join(dir_path, 'VNTC-master/Data/27Topics/Ver1.1/new test')]

# dirs = []
# for path in train_paths:
#     for p in os.listdir(path):
#         dirs.append(os.path.join(path, p))

# for d in tqdm(dirs):
#     sens = get_data(d)
#     sentences = sentences + sens

# # with Pool(8) as pool:
# #     pool.map(get_data, tqdm(dirs))



In [6]:
# pickle.dump(sentences, open('./sentences.pkl', 'wb'))
sentences = pickle.load(open('./sentences.pkl', 'rb'))

In [7]:
def get_corpus(sentences):
    corpus = []
    
    for i in tqdm(range(len(sentences))):
        sen = sentences[i]
        
        words = sen.split(' ')
        tagged_document = gensim.models.doc2vec.TaggedDocument(words, [i])
        
        corpus.append(tagged_document)
        
    return corpus

In [8]:
train_corpus = get_corpus(sentences)

100%|██████████| 2385532/2385532 [00:46<00:00, 50865.69it/s]


In [16]:
from sklearn.utils import shuffle

train_corpus = shuffle(train_corpus)

#### Build Doc2Vec model

In [86]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)
model.build_vocab(train_corpus[:50000])

In [87]:
%time model.train(train_corpus[:50000], total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 3min 55s, sys: 46.1 s, total: 4min 41s
Wall time: 2min 46s


In [88]:
model.infer_vector(train_corpus[100000].words)

array([-0.01640272,  0.30293402, -0.02834823,  0.13452004, -0.07659013,
       -0.33132985, -0.00480685,  0.2618827 , -0.48799312, -0.21672371,
        0.10400254, -0.13873473, -0.10735019, -0.10271706,  0.16801126,
        0.33499536, -0.25886083,  0.07081345,  0.264569  ,  0.13980535,
        0.26452696,  0.09870374, -0.5529773 ,  0.3120131 ,  0.0372953 ,
        0.07732704, -0.03702435, -0.15923078,  0.01857543,  0.04388041,
       -0.33324614,  0.3492984 ,  0.05055666,  0.01133657,  0.16621178,
       -0.19090281,  0.00738728, -0.23280129,  0.28164917,  0.28299242,
       -0.61318856,  0.30712935,  0.25316742,  0.00267887,  0.27907962,
       -0.21710835, -0.13867892, -0.05219948,  0.29381886, -0.32117757,
        0.29475835,  0.17440887, -0.14471418,  0.09133102,  0.11118025,
       -0.09764334, -0.11072824,  0.00535384, -0.06201801,  0.73688567,
        0.391858  , -0.2384683 ,  0.12680122, -0.30809587,  0.1182595 ,
       -0.16347682,  0.03030134, -0.16919139, -0.15018974,  0.54

#### Test with new document

In [89]:
def get_list_sentence_vectors_from_document(doc, model):
    vectors = []
    sens = doc.split('.')
    for sen in sens:
        if len(sen) > 10:
            sen = gensim.utils.simple_preprocess(sen)
            sen = ' '.join(sen)
            sen = ViTokenizer.tokenize(sen)
            sen = sen.split(' ')
            vec = model.infer_vector(sen)
            
            vectors.append(vec)
    
    return np.array(vectors), sens

In [102]:
doc = 'Công sở đậm dư âm Tết trong ngày đầu làm việc. Trong lịch công tác của các bộ ngành 3 ngày đầu năm hầu như không có những cuộc họp. Chương trình làm việc chính của các lãnh đạo là chúc Tết. Một số nơi còn ghi lịch làm việc: lãnh đạo giải quyết công việc thường xuyên tại cơ quan. 9h sáng, hội trường Bộ Giáo dục và Đào tạo rộn tiếng cười đùa, chúc tụng, tiếng ly rượu cụng nhau chan chát. Sau đó các vụ, phòng bắt đầu những cuộc gặp gỡ riêng. Buổi "làm việc" đầu năm kết thúc sớm, với bữa tiệc tân niên tại gia. "Ngày thường, cùng cơ quan đấy nhưng mấy khi có dịp đến nhà nhau. Năm qua, người thì xây nhà mới, người thì có con đầu lòng. Đầu xuân, mọi người trong phòng đến nhà nhau, vừa chúc Tết, thăm hỏi luôn", anh Tuấn, cán bộ Tổng công ty Dệt may Việt Nam tâm sự.  Sau 2 màn tiệc ngọt tại cơ quan, 10h sáng, các thành viên trong phòng của Tuấn "đóng cửa" bắt đầu những cuộc viếng thăm truyền thống. "Hôm nay ai có việc gấp quá thì phải làm nhưng ít người như thế lắm. Cả sếp và nhân viên công ty tôi đều đi chúc Tết. Chúc Tết các phòng xong, chúng tôi đang triệu nhau đến thăm nhà mấy người đồng nghiệp, bạn bè gần đây", chị Nguyễn Thị Bích Thư, phòng Kỹ thuật, Công ty Thép Miền Nam, 56 Thủ Khoa Huân, quận 1 (TP HCM) hoan hỷ. Rộn ràng trong công sở, không khí xuân còn tràn ngập quanh các hàng quán cà phê. 9h sáng, nhưng các hàng quán trên đường Lý Tự Trọng, quận 1 (TP HCM) vẫn khá đông công chức. Họ kể chuyện chơi Tết, chúc tụng, trao lì xì cho nhau và vui vẻ cười đùa. Anh Hoàng, cán bộ một sở trên đường Lý Tự Trọng cho biết, ngày đầu năm, lãnh đạo sở gặp mặt lãnh đạo các phòng, ban, nhân viên, ăn kẹo, uống chút bia thân mật để động viên, lấy khí thế làm việc cho cả năm. Dư vị Tết có lẽ phải kéo dài thêm vài ngày nữa. Trao đổi với VnExpress, Chánh Văn phòng của một bộ cho rằng: Đầu năm chơi nhiều hơn làm đã thành lệ khó sửa. Lãnh đạo cơ quan biết nhưng cũng phải thông cảm. Anh em mời nhau đến nhà chơi, đi làm muộn một chút mình cũng phải thông cảm, quy định cứng nhắc quá thì mất vui. Tuy nhiên, công việc cơ quan vẫn phải đảm bảo, ông này nói. Cũng có một thực tế là đầu năm, người dân cũng mải vui Tết, chưa đến các cơ quan công quyền. Do vậy, không tạo áp lực công việc đối với công chức. Phòng công chứng số 1 (Hà Nội) ngày thường đông nghịt khách nhưng sáng nay vắng hoe. Anh Nguyễn Chí Thiện, công chứng viên cho biết, cả sáng chỉ có khoảng 30 hồ sơ công chứng, thấp kỷ lục trong năm. Vắng khách, sẵn mứt Tết tồn đọng, nhân viên quây quần ngồi uống nước, bàn chuyện du xuân. 11h trưa, nhiều công sở ở Hà Nội khá im ắng, các quán ăn thì đông nghẹt khách. Sáng đi được một tour rồi, trưa tụ họp ở quán ăn lấy sức. Chiều đi vài nhà nữa rồi karaoke, Ngọc Linh, nhân viên kinh doanh một hãng ôtô lớn hào hứng. Với nhiều công chức, ngày đầu năm đi làm còn. . vui hơn Tết!'

sen_vectors, sens = get_list_sentence_vectors_from_document(doc, model=model)

In [103]:
sen_vectors.shape

(33, 300)

In [104]:
X = sen_vectors

In [105]:
from sklearn.cluster import KMeans

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(X)

In [106]:
from sklearn.metrics import pairwise_distances_argmin_min

avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = [sens[closest[idx]] for idx in ordering]

for sen in summary:
    print(sen)

 Đầu xuân, mọi người trong phòng đến nhà nhau, vừa chúc Tết, thăm hỏi luôn", anh Tuấn, cán bộ Tổng công ty Dệt may Việt Nam tâm sự
 9h sáng, hội trường Bộ Giáo dục và Đào tạo rộn tiếng cười đùa, chúc tụng, tiếng ly rượu cụng nhau chan chát
 
 Anh Nguyễn Chí Thiện, công chứng viên cho biết, cả sáng chỉ có khoảng 30 hồ sơ công chứng, thấp kỷ lục trong năm
 Vắng khách, sẵn mứt Tết tồn đọng, nhân viên quây quần ngồi uống nước, bàn chuyện du xuân


## Text Rank